## DDPM 기초 구현

### 구현해야될 항목
  - 각종 변수들 ($\alpha, \tilde\alpha, \mu, ...$)
  - 훈련 코드
  - 샘플링 코드

## 참조
참고한 코드 출처: https://github.com/CodingVillainKor/SimpleDeepLearning/blob/main/DDPM_notebook.ipynb 

위 레포지토리의 원본 구현을 참고하여 수정하였습니다. 

### 라이브러리

In [2]:
# torch
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
import math

# dataset
from torchvision.datasets import CIFAR10
from torchvision import transforms

# check cuda
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    torch.device("cpu")
print(" Device: ", device)

 Device:  cuda:0


### 기본 계수 코드
<details>
  <summary> torch 예제 코드 (cumprod, pad 함수) </summary>
    
1. cumprod 함수 예제
    
    ``` python
    
    a = torch.tensor([5, 7, 10])
    torch.cumprod(a, dim = 0)
    # 출력: tensor([  5,  35, 350])
    
    ```

---

2. pad 함수 예제

    ```python
    a = torch.tensor([1,2,3,4])
    
    # 앞쪽에 2개, 뒤쪽에 3개 패딩 추가 , 기본 값 0 -> 9
    F.pad(a, (2, 3), value = 9)
    # 출력:: tensor([9, 9, 1, 2, 3, 4, 9, 9, 9])
    ```
</details>

- 기본 계수들은 $T$ 크기만큼 계산이 완료가 되어있는 tensor의 상태로 생각하면 된다.

In [4]:
# time t
T = 1000 

# beta: linear하게 증가
betas = torch.linspace(1e-4, 0.02, T).to(device) 

# alpha는 beta의 변형이므로
alphas = 1 - betas

# alpha bar는 alhpa의 누적 합 -> torch의 cumprod 함수
alphas_bar = torch.cumprod(alphas, dim = 0, ).to(device)

# alpha bar 의 t-1도 sampling 과정에서 필요하다. 
# 맨 처음에 alpha가 하나도 없었다는 뜻으로 맨 앞에 1을 추가.
alphas_bar_prev = F.pad(alphas_bar[:-1], (1, 0), value = 1)

#training에 필요한 변수
sqrt_alphas_bar = torch.sqrt(alphas_bar).to(device)
sqrt_one_minus_alphas_bar = torch.sqrt(1. - alphas_bar).to(device)

# sampling에 필요한 변수
reciprocal_alphas_sqrt = torch.sqrt(1. / alphas_bar).to(device)
reciprocal_alphasm1_sqrt = torch.sqrt(1. / alphas_bar - 1.).to(device)
posterior_mean_coef1 = torch.sqrt(alphas_bar_prev) * betas / (1. - alphas_bar).to(device)
posterior_mean_coef2 = torch.sqrt(alphas) * (1. - alphas_bar_prev) / (1. - alphas_bar).to(device)
sigmas = (betas * (1. - alphas_bar_prev) / (1. - alphas_bar)).to(device) # 시그마는 betas를 사용해도 무관함

### 훈련 코드

1. **repeat**:
   - \($ \mathbf{x}_0 \sim q(\mathbf{x}_0) $\)  (데이터 분포에서 샘플링)
   - \($ t \sim \text{Uniform}(\{1, \dots, T\})$ \)  (랜덤한 시간 스텝 샘플링)
   - \($ \epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ \)  (정규분포에서 노이즈 샘플링)
   - Gradient descent step on:

   - $
     \nabla_\theta \left\| \epsilon - \epsilon_\theta \left( \sqrt{\bar{\alpha}_t} \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_t} \epsilon, t \right) \right\|^2
     $

2. **until converged**

<details>
  <summary> torch 예제 코드 (gather, view 함수) </summary>

1. gather 함수
    
    ```python
        # 원본 텐서
        coeff = torch.tensor([10, 20, 30, 40, 50])
        
        # 특정 인덱스를 가져올 t 텐서
        t = torch.tensor([0, 2, 4])  # 인덱스 0, 2, 4를 가져오도록 지정
        
        # gather 사용 (dim=0)
        coeff_t = torch.gather(coeff, index=t, dim=0)
        # 출력: tensor([10, 30, 50])
    ```
    
2. view 함수

    ```python
    # 예제 텐서
    coeff_t = torch.tensor([1, 2, 3, 4])  # Shape: (4,)
    
    # 변환할 차원 리스트
    dims = [2, 2]  # 여기서 len(dims) = 2
    
    # 새로운 차원으로 변환
    B = coeff_t.shape[0] # 4
    reshaped_tensor = coeff_t.view([B] + [1] * len(dims))
    # 출력: torch.Size([4, 1, 1])
    ```
</details>

In [ ]:
def gather_and_expand(coeff, t, xshape):
    ''' 
    t시간에 해당하는 계수(인덱스)를 계수 텐서(coeff)에서 가져오고, 
    해당 계수들을 batch size에 맞게 확장하는 함수 
    '''
    # 입력 텐서의 차원 분리
    batch_size, *dims = xshape # batch_size는 첫 번째 차원, dims는 나머지 차원

     # t시간에 해당하는 계수 가져오기
    coeff_t = torch.gather(coeff, index = t, dim = 0) # Shape: (len(t), ) = (batch_size, )
    
    # 차원 확장
    coeff_t = coeff_t.view([batch_size] + [1] * len(dims)) # 나머지 차원들에 맞게 확장 -> 이후의 계산 차원을 맞추기 위하여
    return coeff_t
    
def train(model, x_0): # x_0: 데이터 분포에서 샘플링한 입력 데이터
    # 랜덤한 시간 스텝에서 배치 사이즈(x_0.shape[0])만큼 샘플링
    t = torch.randint(T, size = (x_0.shape[0], ), device = x_0.device)\
    
    # 정규분포에서 노이즈 샘플링, shape이 batch와 같게
    eps = torch.randn_like(x_0)
    
    # model input, batch 들간 계수
    # 모든 batch가 같은 t를 사용하지 않기 때문에 해당 함수를 사용해야함
    x_t = gather_and_expand(sqrt_alphas_bar, t, x_0.shape)*x_0 + gather_and_expand(sqrt_one_minus_alphas_bar, t, x_0.shape) * eps
    
    # eps와 model output mse 구하기
    loss = F.mse_loss(model(x_t, t), eps)
    return loss

### 샘플링 코드
1. **Initialize**:  
       - \($ \mathbf{x}_T \sim \mathcal{N}(\mathbf{0}, \mathbf{I}) $\)  (시작 샘플은 표준 정규분포에서 샘플링)
    
2. **For \( t = T, ..., 1 \) do**:
   - \($ \mathbf{z} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ \) if \($ t > 1 $\), else \($ \mathbf{z} = 0 $\)  
     (마지막 단계가 아니면 가우시안 노이즈 추가)

   - 업데이트:
     $
     \mathbf{x}_{t-1} = \frac{1}{\sqrt{\alpha_t}} 
     \left( \mathbf{x}_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \epsilon_\theta(\mathbf{x}_t, t) \right) 
     + \sigma_t \mathbf{z}
     $
       - 구현 시엔 $\mu_\theta(x_t,t) = \tilde{\mu}_t \left( \mathbf{x}_t, \frac{1}{\sqrt{\bar{\alpha}_t}} \left( \mathbf{x}_t - \sqrt{1 - \bar{\alpha}_t} \epsilon_{\theta} (\mathbf{x}_t) \right) \right)$ 의 수식을 사용하여 구현한다.
       - 따라서 다음과 같아서 $ \tilde{\mu}_t (\mathbf{x}_t, \mathbf{x}_0) := 
\frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1 - \bar{\alpha}_t} \mathbf{x}_0 
+ \frac{\sqrt{\alpha_t} (1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} \mathbf{x}_t$ , $x_0$은 다음과 같아진다 $x_0 = \frac{1}{\sqrt{\bar{\alpha}_t}} 
\left( \mathbf{x}_t - \sqrt{1 - \bar{\alpha}_t} \epsilon_{\theta} (\mathbf{x}_t) \right)$

3. **Return** \( $\mathbf{x}_0 $\)  (최종 생성된 샘플)

In [ ]:
#기존
def sample(model, x_T): # x_T: noisy data
    x_t = x_T
    for time_step in reversed(range(T)): # T , ..., 1 반복 수행
        # 각 time_step를 batch size로 확장
        t = torch.full((x_T.shape[0], ), time_step, dtype=torch.long, device=device)
        
        # 마지막 단계가 아니면 가우시안 노이즈 샘플링
        z = torch.randn_like(x_t) if time_step else 0

        # 업데이트 과정
        eps = model(x_t, t) # 모델 예측

        # x_0을 구함
        print(f"기존 차원: {gather_and_expand(reciprocal_alphas_sqrt, t, eps.shape).shape} \n  \
                view 차원: {reciprocal_alphas_sqrt[t].view(eps.shape).shape}")
              
        x0_predicted = gather_and_expand(reciprocal_alphas_sqrt, t, eps.shape) * x_t - \
            gather_and_expand(reciprocal_alphasm1_sqrt, t, eps.shape) * eps
        
        # 위 x_0과 함께 평균을 구한다
        mean = gather_and_expand(posterior_mean_coef1, t, eps.shape) * x0_predicted + \
            gather_and_expand(posterior_mean_coef2, t, eps.shape) * x_t
        
        # 분산 구함
        var = torch.sqrt(gather_and_expand(sigmas, t, eps.shape)) * z

        x_t = mean + var
    x_0 = x_t
    return x_0